In [ ]:

import os
import openai
from openai import OpenAI
import json
from pykrx import stock
from pykrx import bond

client = OpenAI()
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_stock_data(code, startdate, enddate):
    df = stock.get_market_ohlcv_by_date(startdate, enddate, code)
    df = df.reset_index()

    csv_data = df[['날짜', '시가', '고가', '저가', '종가']].to_string(index=False)
    return csv_data


messages = [{"role": "user", "content": "삼성전자(005930)의 6월1일부터 6월5일까지 주가 알려줘."}]

functions = [
    {
        "name": "get_stock_data",
        "description": "특정 주식의 특정 기간 주가를 가져옵니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "code": {
                    "type": "string",
                    "description": "검색할 종목의 티커코드 예) 005930",
                },
                "startdate": {
                    "type": "string",
                    "description": "검색기간시작일자. 예) 20250701",
                },
                "enddate": {
                    "type": "string",
                    "description": "검색기간끝일자. 예) 20250701",
                },
            },
            "required": ["code", "startdate", "enddate"],
        },
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call="auto",
    )
response_message = response.choices[0].message
print(response_message)

if response_message.function_call is not None:
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_stock_data": get_stock_data,
    }
    function_name = response_message.function_call.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(
        code=function_args['code'],
        startdate=function_args['startdate'],
        enddate=function_args['enddate']
    )

    messages.append(response_message)
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )
    second_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )  # get a new response from GPT where it can see the function response

    print(second_response.choices[0].message.content)



ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"code":"005930","startdate":"20230601","enddate":"20230605"}', name='get_stock_data'), tool_calls=None)
죄송하지만, 2023년 6월 1일부터 6월 5일까지 삼성전자의 주가 정보를 제공할 수 없습니다. 제공된 데이터는 2025년의 주가 데이터입니다. 최신 주가 정보를 확인하시려면 금융 뉴스 웹사이트나 주식 거래 플랫폼을 이용하시기 바랍니다.
